In [0]:
!pip install wandb
!wandb init

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import subprocess
import os

import wandb
from wandb.keras import WandbCallback

In [2]:
run = wandb.init()
config = run.config
config.img_size = 100
config.batch_size = 128
config.epochs = 50
config.team = "Make_Simpsons_Great_Again"

if config.team is None:
    raise ValueError("You must set config.team on line 16!")

# download the data if it doesn't exist
if not os.path.exists("simpsons"):
    print("Downloading Simpsons dataset...")
    subprocess.check_output("curl https://storage.googleapis.com/wandb-production.appspot.com/mlclass/simpsons.tar.gz | tar xvz", shell=True)

# this is the augmentation configuration we will use for training
# see: https://keras.io/preprocessing/image/#imagedatagenerator-class
train_datagen = ImageDataGenerator(
   rescale=1./255,
   rotation_range=40,
   width_shift_range=0.2,
   height_shift_range=0.2,
   shear_range=0.2,
   zoom_range=0.2,
   horizontal_flip=True,
   fill_mode='nearest')

#print(train_datagen.type)

# only rescaling augmentation for testing:
test_datagen = ImageDataGenerator(rescale=1./255)

#print(test_datagen.type)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
    'simpsons/train',  # this is the target directory
    target_size=(config.img_size,config.img_size),
    batch_size=config.batch_size)

#print(train_generator[1])

# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        'simpsons/test',
        target_size=(config.img_size,config.img_size),
        batch_size=config.batch_size)

#print(test_generator[1])

labels = list(test_generator.class_indices.keys())

#print(labels)
#exit()

W&B Run: https://app.wandb.ai/terwilliker/test/runs/e95ueczr
Call `%%wandb` in the cell containing your training loop to display live results.
Found 14937 images belonging to 13 classes.
Found 1656 images belonging to 13 classes.


In [6]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=(config.img_size, config.img_size, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(80, (3,3), input_shape=(config.img_size, config.img_size, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(96, (3,3), input_shape=(config.img_size, config.img_size, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(112, (3,3), input_shape=(config.img_size, config.img_size, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3,3), input_shape=(config.img_size, config.img_size, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(13, activation="softmax"))
model.compile(optimizer=optimizers.Adam(),
              loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 100, 100, 64)      1792      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 80)        46160     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 80)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 25, 25, 80)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 96)        69216     
__________

In [7]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=config.epochs,
        workers=4,
        validation_data=test_generator,
        callbacks=[WandbCallback(data_type="image", labels=labels, generator=test_generator, save_model=False)],
        validation_steps=len(test_generator))

model.save("trainweights")

Epoch 1/50
117/117 [==============================] - 62s 530ms/step - loss: 2.4802 - acc: 0.1487 - val_loss: 2.3307 - val_acc: 0.2053
Resuming run: https://app.wandb.ai/terwilliker/test/runs/e95ueczr
Epoch 2/50
117/117 [==============================] - 62s 528ms/step - loss: 2.1973 - acc: 0.2473 - val_loss: 1.9861 - val_acc: 0.3508
Epoch 3/50
117/117 [==============================] - 63s 540ms/step - loss: 1.9041 - acc: 0.3574 - val_loss: 1.6952 - val_acc: 0.4469
Epoch 4/50
117/117 [==============================] - 56s 475ms/step - loss: 1.6630 - acc: 0.4524 - val_loss: 1.3593 - val_acc: 0.5707
Epoch 5/50
117/117 [==============================] - 67s 572ms/step - loss: 1.4295 - acc: 0.5320 - val_loss: 1.1803 - val_acc: 0.6178
Epoch 6/50
117/117 [==============================] - 66s 564ms/step - loss: 1.2824 - acc: 0.5848 - val_loss: 0.9995 - val_acc: 0.6969
Epoch 7/50
117/117 [==============================] - 55s 470ms/step - loss: 1.1039 - acc: 0.6428 - val_loss: 0.8131 - val_a